In [39]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import warnings

In [40]:
with open('strokes_split_data_partially_smoted.pkl', 'rb') as f:
    df = pickle.load(f)

X_train = df["X_train_partially_resampled"]
Y_train = df["Y_train_partially_resampled"]

In [41]:
' ----------------------Steuerung: verwendete Spalten --------------------'
drop_columns = [
    #  'age',
    'gender',
     'hypertension',
     'heart_disease',
     'Residence_type',
     'avg_glucose_level',
    'bmi',
    'smoking_status',
     'age_above_60'
    , 'high_glucose'
    , 'did_smoke'
    , 'heart_risk'
    #, 'at_least_one_risk'
    , 'at_least_one_risk_and_high_age'
    , 'all_risks'
    , 'risk_sum'
    ]
X_train = X_train.drop(columns=drop_columns)

In [42]:
X_train.keys()

Index(['age', 'at_least_one_risk'], dtype='object')

In [43]:
' ################# Hier Parameter initiale Modellsteuerung setzen!! ####################'
print("----------------------- Modellparameter ----------------------")
classweight_y = 10
gamma = 0.1
' ############# Modellinstanz #########'
svc = SVC(C=1.0  
          , kernel='rbf'  
          , degree=3   
          , gamma= gamma
          , coef0=0.0  
          , shrinking=True  
          , probability=True 
          , tol=0.001 
          , cache_size=200  
          , class_weight={0: 1, 1: classweight_y}  
          , verbose=False 
          , max_iter=-1  
          , decision_function_shape='ovo'  
          , break_ties=False 
          , random_state=42)

----------------------- Modellparameter ----------------------


In [44]:
' ############# scaling ##################'
scaler = StandardScaler()
' ######################## Pipeline (Scaler + model) für Gridsearch #################'
' ---------------- jetzt Pipeline mit Scaler und Modell erstellen -----'
svc_pipeline = Pipeline([('scaler', StandardScaler()), ('svc', svc)])

In [47]:
############# scope of gridsearch einstellen ###############
scope_of_gridsearch = "finer_and_risk_patients"
############################################################
if scope_of_gridsearch == "rough": ####### liefert große Fläche ##########
    # Liste mit Klassengewichtungen erzeugen
    y_weights = list(range(1, 3, 1))
    list_weight_classes = [{0: 1, 1: value} for value in y_weights]
    list_of_C_values = np.arange(0.005, 1.05, 0.1) # weniger feines C-grid, erste Berechnung
    list_of_gamma_values = [0.01,0.05,0.1, 0.15,0.2]   # weniger feiner gamma-grid, zweite berechnung
    grid_search_name = "gridsearch_rough_partial_smote.pkl"
elif scope_of_gridsearch == "finer_and_risk_patients": ############### verfeinert und age + column-selection at_least_one_risk ###
    # Liste mit Klassengewichtungen erzeugen
    y_weights = list(range(1, 5, 1))
    list_weight_classes = [{0: 1, 1: value} for value in y_weights]
    list_of_C_values = np.arange(0.005, 2.05, 0.1) # weniger feines C-grid, erste Berechnung
    list_of_gamma_values = [0.01,0.05,0.1, 0.125, 0.15,0.17, 0.2, 0.3, 0.5, 0.75, 1]   # weniger feiner gamma-grid, zweite berechnung
    grid_search_name = "gridsearch_rough_partial_smote_risk_patients.pkl"

print(len(list_weight_classes))
print(len(list_of_C_values))
print(len(list_of_gamma_values))
grid_parameters = {"svc__C": list_of_C_values
              , "svc__gamma": list_of_gamma_values  # relevant. Große werte machen kleine Bereiche und andersrum!
              , "svc__class_weight": list_weight_classes}

4
21
11


In [48]:
gridsearch = GridSearchCV(svc_pipeline
                          , grid_parameters
                          , scoring=["recall", "precision"]
                          , cv=6
                          , return_train_score=True
                          , refit ="recall")

In [49]:
# Warnungen ignorieren
warnings.filterwarnings("ignore")

In [50]:
print(datetime.now())

2025-01-14 13:34:36.411169


In [51]:
gridsearch.fit(X_train, Y_train)

GridSearchCV(cv=6,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc',
                                        SVC(class_weight={0: 1, 1: 10},
                                            decision_function_shape='ovo',
                                            gamma=0.1, probability=True,
                                            random_state=42))]),
             param_grid={'svc__C': array([0.005, 0.105, 0.205, 0.305, 0.405, 0.505, 0.605, 0.705, 0.805,
       0.905, 1.005, 1.105, 1.205, 1.305, 1.405, 1.505, 1.605, 1.705,
       1.805, 1.905, 2.005]),
                         'svc__class_weight': [{0: 1, 1: 1}, {0: 1, 1: 2},
                                               {0: 1, 1: 3}, {0: 1, 1: 4}],
                         'svc__gamma': [0.01, 0.05, 0.1, 0.125, 0.15, 0.17, 0.2,
                                        0.3, 0.5, 0.75, 1]},
             refit='recall', return_train_score=True,
             scoring=['recall', 'precision'])

In [52]:
print(datetime.now())

2025-01-14 15:17:33.907688


In [53]:
############## Ergebnisse rausziehen #############
final_model=gridsearch.best_estimator_
cv_res = pd.DataFrame(gridsearch.cv_results_)
cv_res_important_values = cv_res[["mean_test_recall", "mean_train_recall", 'mean_test_precision','mean_train_precision',"param_svc__C", "param_svc__gamma", "param_svc__class_weight"]]
pd.set_option('display.max_rows', None)  # Zeilen: Zeige alle
pd.set_option('display.max_columns', None)  # Spalten: Zeige alle
ergebnisse_sortiert = cv_res_important_values.sort_values(by="mean_test_recall", ascending=False)
print(grid_search_name)

gridsearch_rough_partial_smote_risk_patients.pkl


In [54]:
############ Ergebnisse aktuell ausgewählte Gridsearch abspeichern ###########
new_grid_search = {'Ergebnisse': ergebnisse_sortiert,
                'final_model': final_model,
                    "grid_parameters": grid_parameters}
with open(grid_search_name, 'wb') as data:
    # Einpacken von allen vorbereiteten Daten
    pickle.dump(new_grid_search, data, pickle.HIGHEST_PROTOCOL)

# Einzelstücke verkleben

In [113]:
# # #################### oben neu erzeugte Ergebnisse öffnen ########################
# with open(grid_search_name, 'rb') as f:
#     print(grid_search_name)
#     df = pickle.load(f)
# ergebnis = df["Ergebnisse"]
# # ######################## ältere gridsearchergebnisse öffnen ##########
# with open("gridsearch_tradeoff_final.pkl", 'rb') as f:
#     df = pickle.load(f)
# ergebnis = df["Ergebnisse"]
# # ergebnis1.sort_values(by=["mean_train_recall","mean_test_recall"], ascending=[False, False]).head(3)

gridsearch_C_and_gamma_close_to_zero.pkl


In [114]:
# ############### alles zusammenhängen ranhängen #############
# ergebnis_combined = pd.concat([ergebnis, ergebnis1, ergebnis2, ergebnis3])
# ergebnis_combined = ergebnis_combined.sort_values(by=["mean_train_recall","mean_test_recall"], ascending=[False, False])

# save_merged_results = 0
# if save_merged_results ==1:
#     combined_grid_search = {'Ergebnisse': ergebnis_combined,
#                     'final_model': final_model,
#                         "grid_parameters": grid_parameters}
#     grid_search_name = "gridsearch_complete_tradeoff_area_final.pkl"
#     with open(grid_search_name, 'wb') as data:
#         # Einpacken von allen vorbereiteten Daten
#         pickle.dump(combined_grid_search, data, pickle.HIGHEST_PROTOCOL)